In [1]:
!pip install gym

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
from enum import Enum

In [3]:
def swap(state, action):  
    if action == 0: 
        row0, col0, row1, col1 = 0, 0, 0, 1
    elif action == 1: 
        row0, col0, row1, col1 = 0, 1, 0, 2
    elif action == 2:
        row0, col0, row1, col1 = 0, 0, 1, 0
    elif action == 3:
        row0, col0, row1, col1 = 0, 0, 1, 1
    elif action == 4:
        row0, col0, row1, col1 = 0, 0, 1, 2
    elif action == 5:
        row0, col0, row1, col1 = 1, 0, 1, 1
    elif action == 6:
        row0, col0, row1, col1 = 1, 1, 1, 2
    elif action == 7:
        row0, col0, row1, col1 = 1, 0, 2, 0
    elif action == 8:
        row0, col0, row1, col1 = 1, 0, 2, 1
    elif action == 9:
        row0, col0, row1, col1 = 1, 0, 2, 2
    elif action == 10:
        row0, col0, row1, col1 = 2, 0, 2, 1
    elif action == 11:
        row0, col0, row1, col1 = 2, 1, 2, 2
        
    #swap for all future states
    for i in range(len(state)):
        state[i][row0][col0], state[i][row1][col1] = state[i][row1][col1], state[i][row0][col0]

In [4]:
def getNeighbors(state, row_number, column_number):
    a = [[state[i][j] if  i >= 0 and i < len(state) and j >= 0 and j < len(state[0]) else 0
                    for j in range(column_number-1, column_number+2)]
                        for i in range(row_number-1, row_number+2)]
    return [a[0][1], a[1][0], a[1][2], a[2][1]]

In [5]:
#                      [[1,0,0],
#Takes in a state like  [1,0,2], and checks if the corresonding numbers are neighbors
#                       [2,0,0]]

def isExecutableState(state):
    for row in range(len(state[0])):
        for col in range(len(state[0][0])):
            if state[0][row][col] != 0:
                if not state[0][row][col] in getNeighbors(state[0], row, col):
                    return False
    #return bool
    return True

testing of the functions isValidState and swap

In [6]:
arr = np.zeros((10,3,3), dtype=int)
arr[0][0][1] = 1
arr[0][0][2] = 2
arr[0][1][1] = 1
arr[0][1][2] = 2
arr[1] = arr[0]

print(arr[0])
print(arr[1])
print(isExecutableState(arr))

swap(arr, 1)
print(arr[0])
print(arr[1])
print(isExecutableState(arr))

[[0 1 2]
 [0 1 2]
 [0 0 0]]
[[0 1 2]
 [0 1 2]
 [0 0 0]]
True
[[0 2 1]
 [0 1 2]
 [0 0 0]]
[[0 2 1]
 [0 1 2]
 [0 0 0]]
False


In [7]:
def makeState():
    state = np.zeros((10,3,3))
    for i in range(len(state)):
        random = np.random.choice([3,5,7,9])
        matrix = np.ceil(np.arange(random)/2)
        matrix = np.append(matrix, np.zeros(9-random)).reshape((3,3))
        np.random.shuffle(matrix)
        state[i] = matrix
    return state

Test function makeState

In [8]:
makeState()

array([[[2., 2., 0.],
        [0., 0., 0.],
        [0., 1., 1.]],

       [[3., 0., 0.],
        [2., 2., 3.],
        [0., 1., 1.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 1., 1.]],

       [[2., 2., 3.],
        [0., 1., 1.],
        [3., 0., 0.]],

       [[0., 1., 1.],
        [2., 2., 3.],
        [3., 0., 0.]],

       [[2., 2., 0.],
        [0., 0., 0.],
        [0., 1., 1.]],

       [[3., 0., 0.],
        [2., 2., 3.],
        [0., 1., 1.]],

       [[3., 4., 4.],
        [0., 1., 1.],
        [2., 2., 3.]],

       [[0., 1., 1.],
        [3., 0., 0.],
        [2., 2., 3.]],

       [[0., 0., 0.],
        [0., 1., 1.],
        [2., 2., 0.]]])

In [9]:
#Our enviorment
class Kvant(Env):
    def __init__(self):
        #Actions we can take
        self.action_space = Discrete(12)
        
        #
        self.observation_space = Box(low=np.zeros((10,3,3), dtype=int), high=np.zeros((10,3,3), dtype=int)+4)
        
        #The start state
        self.state = makeState()
        
        #timesteps
        self.time_steps = 100
        
    def step(self, action):
        swap(self.state, action)
        
        self.time_steps -= 1
        
        if isExecutableState(self.state):
            reward = 100
        elif self.time_steps <= 0:
            reward = -1
        else:
            reward = -1
            
    
        if isExecutableState(self.state) or self.time_steps <= 0:
            done = True
        else:
            done = False
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        pass
    def reset(self):
        self.state = makeState()
        self.time_steps = 100
        return self.state

In [10]:
env = Kvant()

/home/surface/.local/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [11]:
env.observation_space.sample()

array([[[2.0291672 , 3.451179  , 1.9308457 ],
        [1.5712548 , 2.712582  , 3.24369   ],
        [1.8229924 , 1.3626896 , 1.0025375 ]],

       [[2.4240074 , 0.24153489, 3.9229357 ],
        [3.7666142 , 2.8267999 , 0.56921476],
        [3.223149  , 1.6040857 , 1.1893152 ]],

       [[0.5930312 , 0.12479167, 1.5873402 ],
        [3.7667856 , 0.9337937 , 2.7028925 ],
        [1.183126  , 0.05507701, 3.1100674 ]],

       [[0.41609016, 0.4045397 , 1.1762248 ],
        [1.5074761 , 2.894781  , 0.03579099],
        [2.3231363 , 2.616475  , 3.317063  ]],

       [[2.053975  , 0.14563237, 0.5337283 ],
        [3.3494956 , 1.729773  , 2.0230894 ],
        [3.1933184 , 1.4354279 , 1.044135  ]],

       [[0.02959235, 3.8737469 , 1.5233475 ],
        [3.5253665 , 2.5695734 , 0.84369916],
        [3.6320512 , 2.0370755 , 2.7654948 ]],

       [[0.49898562, 3.1530101 , 1.6688719 ],
        [2.1529713 , 2.4511635 , 0.17585215],
        [1.8415346 , 3.1680315 , 2.4965174 ]],

       [[0.95720065,

In [12]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:100
Episode:2 Score:98
Episode:3 Score:100
Episode:4 Score:98
Episode:5 Score:-100
Episode:6 Score:100
Episode:7 Score:88
Episode:8 Score:100
Episode:9 Score:-100
Episode:10 Score:73
